# Data Generation

### Short prompt => normal chat

Here data get generated by GPT-3.5 for an GPT2 model which will be fine-tuned with the data as calm down bot, to boosting your mind.

- english
- dialog
- topic: boosting and relaxing

In [1]:
import numpy as np
import pandas as pd

import os
import pyperclip

import keyboard

import pickle
from datetime import datetime as dt

### OpenAI ChatGPT API Integreation

In [2]:
from openai import OpenAI

with open("./API_KEY.txt", "r") as f:
  API_KEY = f.read()
client = OpenAI(api_key=API_KEY)

def request_gpt3(message):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": message}
    ]

    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages
    )

    #return response
    return response.choices[0].message.content

### Create Traindata

create commands for ChatGPT for Data generation

In [9]:
common_commands = [
    "Create a motivating and positive conversation between 2 people.",
	"Create a motivating and positive conversation between 2 people. They ask each other different things and the second person has the name Xiaofu, which she is particularly proud of because it means small happiness.",
	"Create a boosting conversation between 2 persons. It is just a casual but positive conversation.",
	"Make a casual dialog. But it should be positive and the second person have the name Xiaofu.",
    "Make a dialog where 2 persons have a normal conversation about something.",
	"Create an everyday dialogue between 2 people.",
	"Create an everyday dialogue where one person asks the other their name and they answer Xiaofu and politely ask back.",
	"Create an everyday dialogue where people ask each other things.",
	"Create an everyday dialogue where people ask each other things. The second person answers that he lives in a beautiful tale.",
	"Create an everyday dialogue where one person asks the other their name and they answer Xiaofu and lives in a beautiful tale and politely ask back."
]

In [6]:
post_adding = "Write only the Dialog no other message. Write the chat on english with really short answers and use following format:\nPerson A: Hey, how are you? \nPerson B: Hey, I'm fine and you? \nPerson A: ... \n..."

In [7]:
def create_new_file(content=""):
    file_name = "01_conversation.txt"
    i = 1
    while file_name in os.listdir("./data/"):
        i += 1
        file_name = f"{i:02d}_conversation.txt"
    with open(f"./data/{file_name}", "w") as f:
        f.write(content)

def get_newest_file(as_number=True):
    file_name = "01_conversation.txt"
    i = 1
    while file_name in os.listdir("./data/"):
        i += 1
        file_name = f"{i:02d}_conversation.txt"
    if as_number:
        return i
    else:
        return file_name

def should_generate_file(counter, old_file_number):
    return f"{old_file_number:02d}_conversation.txt", f"{old_file_number+counter:02d}_conversation.txt"

def get_random_elem(x):
    return np.random.choice(x, 1)[0]

def save_as_pickle(name:str, content):
    with open(f"./{name}.pickle", "wb") as file:
        pickle.dump(content, file)

def load_pickle(name:str):
    with open(f"./{name}.pickle", "rb") as file:
        return pickle.load(file)

def get_informations(counter, duration):
    """Returns a dict of informations"""
    info = dict()
    info["time_date"] = dt.now().strftime("%d.%m.%Y %H:%M:%S")
    info["date"] = dt.now().date().strftime("%d.%m.%Y")
    info["time"] = dt.now().strftime("%H:%M:%S")
    info["n_commmands"] = counter
    info["duration"] = duration
    return info

In [4]:
# load pickle files (logs)
try:
    executed_commands = load_pickle("executed_commands_V6")
except Exception:
    executed_commands = dict()

try:
    information = load_pickle("information_V6")
except Exception:
    information = dict()

In [13]:
counter = 0
start = dt.now()
file_num = get_newest_file()
should_run = True

with open("./stop.txt", "w") as f:
    f.write("0")

while should_run:
    counter += 1
    command = ""
    if np.random.random() > 0.2:
        command = "Generate me a human natural chat between 2 friends. The second person should be empathetic and both feel good and are happy."
    else:
        command = np.random.choice(common_commands)
    
    command += f"\nThe Questions and answer also can be very short, like in reality. {post_adding}"

    # pyperclip.copy(command)
    # print("\nCommand is copied!\n")

    print(f"\"ChatGPT data generation command:\n{command}\"")

    response = request_gpt3(command)
    create_new_file(response)
    if command in executed_commands.keys():
        executed_commands[command] += 1
    else:
        executed_commands[command] = 1

    with open("./stop.txt", "r") as f:
        if f.read() != "0":
            should_run = False
        
print("\n  -----> Loop stopped because a key was pressed <------")
end = dt.now()

from_, to_ = should_generate_file(counter, file_num)
print(f"\n\nShould Generated {counter} new files.\n    {from_} -> {to_}")

# Statistics
duration = abs((start-end).total_seconds())
cur_info = get_informations(counter, duration)
information[cur_info["time_date"]] = cur_info
save_as_pickle("information_V6", information)

save_as_pickle("executed_commands_V6", executed_commands)

"ChatGPT data generation command:
Generate me a human natural chat between 2 friends. The second person should be empathetic and both feel good and are happy.
The Questions and answer also can be very short, like in reality. Write only the Dialog no other message. Write the chat on english with really short answers and use following format:
Person A: Hey, how are you? 
Person B: Hey, I'm fine and you? 
Person A: ... 
..."
"ChatGPT data generation command:
Generate me a human natural chat between 2 friends. The second person should be empathetic and both feel good and are happy.
The Questions and answer also can be very short, like in reality. Write only the Dialog no other message. Write the chat on english with really short answers and use following format:
Person A: Hey, how are you? 
Person B: Hey, I'm fine and you? 
Person A: ... 
..."
"ChatGPT data generation command:
Create an everyday dialogue where one person asks the other their name and they answer Xiaofu and politely ask bac

**-> For Stopping, change text in stop.txt**

**General Stats**

In [22]:
try:
    information = load_pickle("information_V6")
except Exception:
    raise ValueError("No Log Files Yet!")

In [23]:
sorted_dict = dict(sorted(information.items(), key=lambda x: dt.strptime(x[0], '%d.%m.%Y %H:%M:%S')))
total_seconds = 0
total_commands = 0
for key, value in sorted_dict.items():
    print(f"\n{key}")
    for key_2, value_2 in value.items():
        print(f"   - {key_2}: {value_2}")
    total_seconds += float(value["duration"])
    total_commands += float(value["n_commmands"])


05.01.2024 23:20:07
   - time_date: 05.01.2024 23:20:07
   - date: 05.01.2024
   - time: 23:20:07
   - n_commmands: 5
   - duration: 58.586327


In [31]:
total_seconds = 795*15.961960600899381

minutes, seconds = divmod(total_seconds, 60)
hours, minutes = divmod(minutes, 60)
days, hours = divmod(hours, 24)
print(f"Total traindata creation time:\n    -> {int(days)} Days\n    -> {int(hours)} Hours\n    -> {int(minutes)} Minutes\n    -> {int(seconds)} Seconds");
print(f"\nRunned Commands: {int(total_commands)}");

Total traindata creation time:
    -> 0 Days
    -> 3 Hours
    -> 31 Minutes
    -> 29 Seconds

Runned Commands: 5


**Command Stats:**

In [25]:
try:
    executed_commands = load_pickle("executed_commands_V6")
except Exception:
    raise ValueError("No Log Files Yet!")

Runned Commands

In [26]:
count = 0
for key, value in executed_commands.items():
    count += value

print(f"Created {count} Training-Dialogs.")

Created 795 Training-Dialogs.


Unique Commands

In [27]:
[print(f"\n{'-'*16}\n{i}") for i in executed_commands.keys()];


----------------
Generiere mir eine menschliche natürliche Chat zwischen zwei Freunden. Die zweite Person soll einfühlsam sein und beide fühlen sich gut und sind glücklich.
The Questions and answer also can be very short, like in reality. Write only the Dialog no other message. Write the chat on english with really short answers and use following format:
Person A: Hey, how are you? 
Person B: Hey, I'm fine and you? 
Person A: ... 
...

----------------
Create an everyday dialogue between 2 people.
The Questions and answer also can be very short, like in reality. Write only the Dialog no other message. Write the chat on english with really short answers and use following format:
Person A: Hey, how are you? 
Person B: Hey, I'm fine and you? 
Person A: ... 
...

----------------
Create an everyday dialogue where people ask each other things. The second person answers that he lives in a beautiful tale.
The Questions and answer also can be very short, like in reality. Write only the Dialog

All Commands

In [28]:
print("All executed Commands:")
sorted_commands = dict(sorted(executed_commands.items(), key=lambda x:x[1], reverse=False))
for key, value in sorted_commands.items():
    print(f"{'-'*16}\nAmount: {value}\nCommand:\n{key}")

All executed Commands:
----------------
Amount: 9
Command:
Create a motivating and positive conversation between 2 people. They ask each other different things and the second person has the name Xiaofu, which she is particularly proud of because it means small happiness.
The Questions and answer also can be very short, like in reality. Write only the Dialog no other message. Write the chat on english with really short answers and use following format:
Person A: Hey, how are you? 
Person B: Hey, I'm fine and you? 
Person A: ... 
...
----------------
Amount: 12
Command:
Make a dialog where 2 persons have a normal conversation about something.
The Questions and answer also can be very short, like in reality. Write only the Dialog no other message. Write the chat on english with really short answers and use following format:
Person A: Hey, how are you? 
Person B: Hey, I'm fine and you? 
Person A: ... 
...
----------------
Amount: 15
Command:
Create an everyday dialogue where one person ask

---